In [ ]:
# import packages

import numpy as np
np.random.seed(87)

import keras
from keras.datasets import cifar10
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D



In [ ]:
# 新聞標題 (文字應用的NN)
from keras.layers import Input, Dense
from keras.layers import Embedding  # Embedding 層將輸入序列編碼為一個稠密向量的序列
from keras.layers import LSTM       # LSTM 層把向量序列轉換成單個向量，它包含整個序列的上下文信息
from keras.models import Model



In [ ]:
# 設定在訓練模型(fit model)時，所用到的參數
batch_size = 32
num_classes = 10
epochs = 10




"""
學習率太低，需要更新很多次才能到最佳解，
學習率太高，有可能會造成梯度走不進去局部極值(但也可以擺脫局部極值的問題，等等有範例)。


這個學習率太小，初始值不好，解就會掉到局部極小值。
這個學習率(0.0004)對此例子來說，雖然步伐夠大跳出了局部極值，但到全域極值時，因為步伐太大，所以走不到最好的值。
這個學習率(0.0003)對此例子來說就夠了，可以走到全域極值。


### 學習率對梯度下降法的影響

學習率較小時，收斂到正確結果的速度較慢。

學習率較大時，容易在搜索過程中發生震盪。


### Result

學習率較大時，容易在搜索過程中發生震盪，而發生震盪的根本原因無非就是搜索的步長邁的太大了

如果讓能夠lr隨著迭代週期不斷衰減變小，那麼搜索時邁的步長就能不斷減少以減緩震盪學習率衰減因子由此誕生


"""

In [ ]:
# 資料準備 / 取得資料，並分成 Training 與 Test set
(x_img_train,y_label_train), (x_img_test, y_label_test) = cifar10.load_data()

# 確認資料維度
print("train data:",'images:', x_img_train.shape, " labels:", y_label_train.shape) 
print("test  data:",'images:', x_img_test.shape , " labels:", y_label_test.shape) 



# 資料(影像)標準化 (Image normalization)，並設定 data array 為浮點數
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0



# 將資料從圖形 (RGB) 轉為向量 (Single Vector)
x_train = x_train.reshape((len(x_train), -1))
x_test = x_test.reshape((len(x_test), -1))





# 轉換 label 為 OneHot Encoding (只要是做分類問題的時候，都要對 label 做 one-hot encoding喔！) Convert class vectors to binary class matrices.
# 針對 Label 做 ONE HOT Encoding, 並查看維度資訊
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
print(y_label_test_OneHot.shape)







# 是否要做資料處理
if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(x_train, 
                        y_train,
                        batch_size = batch_size,
                        epochs = epochs,
                        validation_data = (x_test, y_test),
                        shuffle = True)
else:
    print('Using real-time data augmentation.')
    print('')
        
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                zca_epsilon=1e-06,  # epsilon for ZCA whitening
                rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
                # randomly shift images horizontally (fraction of total width)
                width_shift_range=0.1,
                # randomly shift images vertically (fraction of total height)
                height_shift_range=0.1,
                shear_range=0.,  # set range for random shear
                zoom_range=0.,  # set range for random zoom
                channel_shift_range=0.,  # set range for random channel shifts
                # set mode for filling points outside the input boundaries
                fill_mode='nearest',
                cval=0.,  # value used for fill_mode = "constant"
                horizontal_flip=True,  # randomly flip images
                vertical_flip=False,  # randomly flip images
                # set rescaling factor (applied before any other transformation)
                rescale=None,
                # set function that will be applied on each input
                preprocessing_function=None,
                # image data format, either "channels_first" or "channels_last"
                data_format=None,
                # fraction of images reserved for validation (strictly between 0 and 1)
                validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    
    """ 這個不是訓練模型 (fit model，底下的 model.fit(...) 才是！) """
    datagen.fit(x_train) # 這個不是訓練模型 (fit model，底下的 model.fit(...) 才是！)
    
    history = model.fit(x_train, 
                        y_train,
                        batch_size = batch_size,
                        epochs = epochs,
                        validation_data = (x_test, y_test),
                        shuffle = True)   

'''
   第四步：訓練
   .fit的一些參數
   batch_size：對總的樣本數進行分組，每組包含的樣本數量
   epochs ：訓練次數
   shuffle：是否把數據隨機打亂之後再進行訓練
   validation_split：拿出百分之多少用來做交叉驗證
   verbose：屏顯模式 - 0：不輸出, 1：輸出進度, 2：輸出每次的訓練結果
''' 

## MLP (多層 Perceptron) 的優缺點

MLP 優點

> 建立 「非線性」 模型、 「real-time」 模型

MLP 缺點

>使用不同的初始權重，會讓驗證時的準確率浮動 MLP 需要調整每層神經元數、層數、迭代次數 對於特徵預處理很敏感

In [ ]:
# 建立模型
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
# build model
"""
NN的組成

1. Input Layer

    Weights

2. Hidden Layer

    Net Input Function

    Activation Function : 給神經元引入非線性因素，使得 神經網路能任意逼近任何非線性函數
    
    
    
    ### 啟動函數
    
    啟動函數的最⼤作⽤就是非線性化．如果不⽤啟動函數的話，無論神經網路有多少層，輸出都是輸入的線性組合

    它應該是可以區分前⾏網路與反向式傳播網路的網路參數更新，然後相應地使⽤梯度下降或任何其他優化技術優化權重以減少誤差



    ### 根據各個函數的優缺點來配置
    
    如果使⽤ ReLU，要⼩⼼設置 learning rate，注意不要讓網路出現很多「dead」 神經元，如果不好解決，可以試試 Leaky ReLU、PReLU 或者Maxout
    
    
    
    ### 根據問題的性質
    
    ⽤於分類器時，Sigmoid 函數及其組合通常效果更好

    由於梯度消失問題，有時要避免使⽤ sigmoid 和 tanh 函數。ReLU 函數是⼀個通⽤的啟動函數，⽬前在⼤多數情況下使⽤

    如果神經網路中出現死神經元，那麼 PReLU 函數就是最好的選擇

    ReLU 函數建議只能在隱藏層中使⽤





3. Output Layer
"""







"""# model - 1"""
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))



"""# model - 2"""
# 宣告採用序列模型
model = Sequential()

# 卷積層1 - filters=32
# 與池化層1
model.add(Conv2D(filters=32,
                 kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 卷積層2 - filters=64
# 與池化層2
model.add(Conv2D(filters=64, 
                 kernel_size=(3, 3), 
                 activation='relu', 
                 padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 卷積層3 - filters=128
# 與池化層3
model.add(Conv2D(filters=128, 
                 kernel_size=(3, 3), 
                 activation='relu', 
                 padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 卷積層4 - filters=256
# 與池化層4
model.add(Conv2D(filters=256, 
                 kernel_size=(3, 3), 
                 activation='relu', 
                 padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

"""
# 建立神經網路(平坦層、隱藏層、輸出層)
"""
""" 1. Flatten layer (Input Layer) """
model.add(Flatten())

""" 2. Fully Connected Layer (Hidden Layer) """
# 建立全網路連接層
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))

""" 3. Output Layer """
#建立輸出層
model.add(Dense(10, activation='softmax'))

#檢查model 的STACK
print(model.summary())












In [ ]:
# 載入之前訓練的模型

try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")







In [ ]:
# compile model (模型編譯)
"""
# 模型編譯

在訓練模型之前，您需要配置學習過程，的英文這通過compile方法完成的它接收三個參數：





1. Optimizer

優化器optimizer。它可以是現有優化器的字符串標識符，如rmsprop或adagrad，也可以是Optimizer類的實例。



### SGD-隨機梯度下降法(stochastic gradient decent)

優點：SGD 每次更新時對每個樣本進⾏梯度更新， 對於很⼤的數據集來說，可能會有相似的樣本，
     ⽽ SGD ⼀次只進⾏⼀次更新，就沒有冗餘，⽽且比較快

缺點：但是 SGD 因為更新比較頻繁，會造成 cost function 有嚴重的震盪
















2. Loss

損失函數的損失，模型試圖最小化的目標函數它可以是現有損失函數的字符串標識符，如。categorical_crossentropy或mse，也可以是一個目標函數


### Gradient 梯度
在微積分裡⾯，對多元函數的參數求 ∂ 偏導數，把求得的各個參數的偏導數以向量的形式寫出來，就是梯度。

最常⽤的優化算法 - 梯度下降
⽬的：沿著⽬標函數梯度下降的⽅向搜索極⼩值（也可以沿著梯度上升的⽅向搜索極⼤值）
要計算 Gradient Descent，考慮
• Loss = 實際 ydata – 預測 ydata



現在分析在梯度下降法中最常聽到的一句話：「梯度下降法就是朝著梯度的反方向迭代地調整參數直到收斂。」這裡的梯度就是 Δ ,而梯度的反方向就是- Δ 的符號方向-- -梯度實際上是個向量。所以這個角度來說，即使我們只有一個參數需要調整，也可以認為它是個一維的向量。整個過程你可以想像自己站在一個山坡上，準備走到山腳下（最小值的地方），於是很自然地你會考慮朝著哪個方向走，方向由 - Δ 的方向給出，而至於一次走多遠，由|α Δ |來控制。這種方式相信你應該能理解其只能找到局部最小值，而不是全局的。








### Gradient Descent 梯度下降法的類型
1. 批次梯度下降(Batch gradient descent)

2. 隨機梯度下降(Stochastic gradient descent)

隨機梯度下降最大的缺點在於每次更新可能並不會按照正確的方向進行，因此可以帶來優化波動(擾動)，如下圖： img3不過從另一個方面來看，隨機梯度下降所帶來的波動有個好處就是，對於類似盆地區域（即很多局部極小值點）那麼這個波動的特點可能會使得優化的方向從當前的局部極小值點跳到另一個更好的局部極小值點，這樣便可能對於非凸函數，最終收斂於一個較好的局部極值點，甚至全局極值點。 由於波動，因此會使得迭代次數（學習次數）增多，即收斂速度變慢。不過最終其會和全量梯度下降算法一樣，具有相同的收斂性，即凸函數收斂於全局極值點，非凸損失函數收斂於局部極值點。

3. 小批量梯度下降(Mini-batch gradient descent)





### 梯度下降法的挑戰

雖然梯度下降算法效果很好，並且廣泛使用，但同時其也存在一些挑戰與問題需要解決：

1. 選擇一個合理的學習速率很難。如果學習速率過小，則會導致收斂速度很慢。如果學習速率過大，那麼其會阻礙收斂，即在極值點附近會振盪。

2. 學習速率調整(又稱學習速率調度，Learning rate schedules)[11]試圖在每次更新過程中，改變學習速率，如退火。一般使用某種事先設定的策        略或者在每次迭代中衰減一個較小的閾值。無論哪種調整方法，都需要事先進行固定設置，這邊便無法自適應每次學習的數據集特點。

3. 模型所有的參數每次更新都是使用相同的學習速率。如果數據特徵是稀疏的或者每個特徵有著不同的取值統計特徵與空間，那麼便不能在每次更新中每        個參數使用相同的學習速率，那些很少出現的特徵應該使用一個相對較大的學習速率。

4. 對於非凸目標函數，容易陷入那些次優的局部極值點中，如在神經網路中。那麼如何避免呢。Dauphin指出更嚴重的問題不是局部極值點，而是鞍點 (These saddle points are usually surrounded by a plateau of the same error, which makes it notoriously hard for SGD to escape, as the gradient is close to zero in all dimensions .)






### avoid local minimum
 • Item-1：在訓練神經網絡的時候，通常在訓練剛開始的時候使⽤較⼤的learning rate，隨著訓練的進⾏，我們會慢慢的減⼩ learning rate
 • Item-2：隨著 iteration 改變 Learning - decay
 • Item-3：momentum(動量)

def GD(w_init, df, epochs, lr):    
    '''  
    梯度下降法。給定起始點與目標函數的一階導函數，求在epochs次反覆運算中x的更新值
        :param w_init: w的init value    
        :param df: 目標函數的一階導函數    
        :param epochs: 反覆運算週期    
        :param lr: 學習率    
        :return: x在每次反覆運算後的位置   
    ''' 

> ### 1. lr

(1)學習率較小時，收斂到極值的速度較慢。

(2)學習率較大時，容易在搜索過程中發生震盪。



> ### 2. Momentum (動量)

「⼀顆球從⼭上滾下來，在下坡的時候速度越來越快，遇到上坡，⽅向改變，速度下降」

加入的這⼀項，可以使得梯度⽅向不變的維度上速度變快，梯度⽅向有所改變的維度上的更新速度變慢，這樣就可以加快收斂並減⼩震盪


(1)(ß 即momentum係數，通俗的理解上面式子就是，如果上一次的momentum（即ß ）與這一次的負梯度方向是相同的，那這次下降的幅度就會加大，所以這樣做能夠達到加速收斂的過程

(2)如果上一次的momentum（即ß ）與這一次的負梯度方向是相反的，那這次下降的幅度就會縮減，所以這樣做能夠達到減速收斂的過程




> ### 3. decay (學習率衰減公式)

lr_i = lr_start 1.0 / (1.0 + decay i)

其中lr_i為第一迭代i時的學習率，lr_start為原始學習率，decay為一個介於[0.0, 1.0]的小數。從公式上可看出：

decay越小，學習率衰減地越慢，當decay = 0時，學習率保持不變。 decay越大，學習率衰減地越快，當decay = 1時，學習率衰減最快






### Back Propadation

其實他就是 Gradient Descent ， 只是在面對「百萬維」的參數量時，計算量更精簡

25:45

從前面往後（左邊往右）算的運算量其實非常大

但如果從最後一層 output layer 的偏微分算回來，因為 Chain Rule 的關係，一切就都萬事俱備、迎刃而解

這就是 Back Propadation，就是建一個反過來方向的 Neural Network 啦





3. metrics 評估指標

評估標準指標。對於任何分類問題，你都希望將其設置為metrics = ['accuracy']。評估標準可以是現有的標準的字符串標識符，也可以是自定義的評估標準函數。
"""


model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])






In [ ]:
# fit(train) model

'''
   第四步：訓練
   .fit的一些參數
   batch_size：對總的樣本數進行分組，每組包含的樣本數量
   epochs ：訓練次數
   shuffle：是否把數據隨機打亂之後再進行訓練
   validation_split：拿出百分之多少用來做交叉驗證
   verbose：屏顯模式 - 0：不輸出, 1：輸出進度, 2：輸出每次的訓練結果
''' 

#模型訓練, "Train_History" 把訓練過程所得到的數值存起來
train_history = model.fit(x_img_train_normalize, y_label_train_OneHot,
                          validation_split=0.25,
                          epochs=12, 
                          batch_size=128, 
                          verbose=1
                         )         

# [validation_split = 0.2] validation_split：在0和1之間浮動。用作驗證數據的訓練數據的比例。
# 該模型將訓練數據的這一部分分開，不會對其進行訓練，並將在每個時期結束時評估該數據的損失和任何模型指標。
# [batch_size]：整數或None。每個梯度更新的樣本數。指定，batch_size為128



#---



model.fit(x_train, y_train, 
          
          epochs = 500, # 請將 Epoch 加到 500 個，並觀察 learning curve 的走勢
          
          batch_size=256, 
          validation_data=(x_test, y_test), 
          shuffle=True)

In [ ]:
# 以視覺畫方式檢視訓練過程

# 畫出 train acc, validation acc，以檢視是否有 overfitting / underfitting

import matplotlib.pyplot as plt
%matplotlib inline

# 定義一個繪圖函數
def show_train_history(train_acc, test_acc):
    plt.plot(train_history.history[train_acc])
    plt.plot(train_history.history[test_acc])
    plt.title('Train History')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train_acc', 'val_acc'], loc='upper left')
    plt.show()
    
show_train_history('acc', 'val_acc')





#---


# 以視覺畫方式檢視訓練過程
import matplotlib.pyplot as plt
%matplotlib inline

train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

#    第六步：輸出
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])



#---

# scores = model.evaluate(x_Test_normalize, y_Test_OneHot)

scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print()
print('accuracy=',scores[1])

In [ ]:
# 結合 compile, fit, plot, scores

def check_model(loss_function) :
        
    model.compile(loss = loss_function, optimizer='sgd', metrics=['accuracy'])

    #模型訓練, "Train_History" 把訓練過程所得到的數值存起來
    train_history = model.fit(x_img_train_normalize, 
                              y_label_train_OneHot,
                              validation_split=0.25,
                              epochs=12, 
                              batch_size=128, 
                              verbose=1
                             )         

    # [validation_split = 0.2] validation_split：在0和1之間浮動。用作驗證數據的訓練數據的比例。
    # 該模型將訓練數據的這一部分分開，不會對其進行訓練，並將在每個時期結束時評估該數據的損失和任何模型指標。
    # [batch_size]：整數或None。每個梯度更新的樣本數。指定，batch_size為128

    # 定義一個繪圖函數
    def show_train_history(train_history, train, validation):
        plt.plot(train_history.history[train])
        plt.plot(train_history.history[validation])
        plt.title('Train History')
        plt.ylabel(train)
        plt.xlabel('Epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()
        
    # plot
    show_train_history(train_history, 'acc', 'val_acc')
    show_train_history(train_history, 'loss', 'val_loss')

    # scores
    scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
    print()
    print('accuracy=',scores[1])
    
    
    
#check_model(loss_function)
check_model('categorical_crossentropy')
check_model('mean_squared_error')

In [1]:
# Tuning

"""

What is “overfitting” ?

(1) Overfitting的意思就是太過追求參數完美預測出訓練數據的結果，反而導致實際預測效果不佳

high variance可以理解為其有著過多的 variable


(2) 跟overfit相反的狀況：underfit，代表在訓練數據中也有著高預測誤差的問題

high bias可以理解為其會過度依賴其截距(θ0)






### 可能會有人好奇 λ 到底代表什麼意思？

λ代表的其實是我們對於預測誤差跟正規項的取捨 !






當今天 λ越大，模型會比較不重視預測落差反而是極力地想要壓低所有θ值的大小 就像是我們如果把 λ 設成10¹⁰的話，所有θ值都會趨近於0，最後形成一條直線的hypothesis img7

而若是 λ 越小甚至趨近於0，可能對改善overfitting就沒什麼太大的幫助了

我在學習正規化的時候，對於為什麼只是加上個正規多項式就可以改善overfitting感到非常疑惑

畢竟我們根本不知道要降低哪一個 θ值(feature)的影響力啊

就這樣直接一視同仁的一起打壓所有的θ值到底為什麼會有效？



這是我想到的答案(沒有證實過)：的確是一視同仁的打壓所有的 θ值，但是若是今天θ值只要設定到某幾個數字的話可以使預測誤差降到非常低的話，那麼模型如果為了貪小便宜的將那幾個 θ值給壓低反而造成了預測誤差的大幅上升，使得J(θ)不降反升的反效果的話，會非常得不償失，因此模型將會斟酌不要降低那些重要的 θ值

上述的假設都建立在今天 λ 設立得宜的情況以及有足夠的資料來support預測落差的下降，說服模型不要把重要的 θ值降低


"""








"""
EliteDataScience - 如何減少 Overfitting 的發生

1. 使用 K-fold cross validation
    
    找到一組參數可以在多組不同的 validation 上得到相似的結果

2. 使用更多的訓練資料

3. 減少 Features (參數) 的使用量 : 人工選擇、model selection algorithm
    
    避免參數比潛在組合更多的狀況發生，以免模型靠硬記就可以得到結果

4. 在模型訓練的過程中加入正則化參數 (Regularization) : 維持現有的features，但是降低部分不重要feature的影響力。這對於有著許多feature的hypothesis很有幫助
    
    控制 input 的改變對模型造成的影響太大。
    
    
    
"""







"""
### Underfitting

Error from Bias

Bias is the difference between your model's expected predictions and the true values.



### Overfitting

Error from Variance

Variance refers to your algorithm's sensitivity to specific sets of training data..




## How to Prevent Overfitting
1. Cross-validation
2. Train with more data
3. Remove features
4. Early stopping
5. Regularization
6. Ensembling

"""

'\n\n\n'